
<div class="elfjS" data-track-load="description_content"><p>Given an <code>n x n</code> binary matrix <code>grid</code>, return <em>the length of the shortest <strong>clear path</strong> in the matrix</em>. If there is no clear path, return <code>-1</code>.</p>

<p>A <strong>clear path</strong> in a binary matrix is a path from the <strong>top-left</strong> cell (i.e., <code>(0, 0)</code>) to the <strong>bottom-right</strong> cell (i.e., <code>(n - 1, n - 1)</code>) such that:</p>

<ul>
	<li>All the visited cells of the path are <code>0</code>.</li>
	<li>All the adjacent cells of the path are <strong>8-directionally</strong> connected (i.e., they are different and they share an edge or a corner).</li>
</ul>

<p>The <strong>length of a clear path</strong> is the number of visited cells of this path.</p>

<p>&nbsp;</p>
<p><strong class="example">Example 1:</strong></p>
<img alt="" src="https://assets.leetcode.com/uploads/2021/02/18/example1_1.png" style="width: 500px; height: 234px;">
<pre><strong>Input:</strong> grid = [[0,1],[1,0]]
<strong>Output:</strong> 2
</pre>

<p><strong class="example">Example 2:</strong></p>
<img alt="" src="https://assets.leetcode.com/uploads/2021/02/18/example2_1.png" style="height: 216px; width: 500px;">
<pre><strong>Input:</strong> grid = [[0,0,0],[1,1,0],[1,1,0]]
<strong>Output:</strong> 4
</pre>

<p><strong class="example">Example 3:</strong></p>

<pre><strong>Input:</strong> grid = [[1,0,0],[1,1,0],[1,1,0]]
<strong>Output:</strong> -1
</pre>

<p>&nbsp;</p>
<p><strong>Constraints:</strong></p>

<ul>
	<li><code>n == grid.length</code></li>
	<li><code>n == grid[i].length</code></li>
	<li><code>1 &lt;= n &lt;= 100</code></li>
	<li><code>grid[i][j] is 0 or 1</code></li>
</ul>
</div>


In [3]:
from typing import *
from math import sqrt
import heapq

In [4]:
class Node:
    def __init__(self, pos: Tuple[int, int], dist: int, n: int):
        self.pos = pos
        self.dist = dist
        self.n = n
        self.visited = False

        # We use the diagonal distance heuristic which is correct for scenarios where the
        # traversal can only be done in 8 directions. If we were limited to 4 directions,
        # we would use manhattan distance. If it were any direction, we would use Euclidean.
        # We assume the side length of a cell is 1 for this calculation (therefore its diagonal
        # length is sqrt(2) - 2).
        dx = self.pos[0] - self.n + 1
        dy = self.pos[1] - self.n + 1
        self.diag_dist = dx + dy + (sqrt(2) - 2)*min(dy, dy)

    def __lt__(self, other):
        # we sort the queue based on the node's current distance from the start and its heuristic
        # distance to the destination
        return (self.dist + self.diag_dist) < (other.dist + other.diag_dist)

class Solution:
    def shortestPathBinaryMatrix(self, grid: List[List[int]], verbose: bool = False) -> int:
        n = len(grid)
        if grid[0][0] == 1 or grid[n-1][n-1] == 1:
            # the start and/or end position is 1, cannot visit it
            return -1
        elif n == 1:
            # grid is a single spot, the path only contains this one element
            return 1

        nodes = [[Node((i,j), -1, n) for j in range(n)] for i in range(n)]
        nodes[0][0].dist = 1
        in_queue = [[0 for _ in range(n)] for _ in range(n)]
        in_queue[0][0] = True
        pq = [nodes[0][0]]

        while len(pq) > 0:
            node = heapq.heappop(pq)
            dist = node.dist
            node.visited = True
            in_queue[node.pos[0]][node.pos[1]] = False

            # iterate through the 8 neighbors (potentially less if some (i,j) pairs are not in range)
            for i in range(max(node.pos[0] - 1, 0), min(node.pos[0] + 2, n)):
                for j in range(max(node.pos[1] - 1, 0), min(node.pos[1] + 2, n)):

                    if (i,j) == (n-1, n-1):
                        # we found the destination, return its distance
                        return dist + 1

                    if grid[i][j] == 0 and (i,j) != node.pos:
                        # the node can be traversed to and it is not our current node

                        if in_queue[i][j]:
                            # this node is in the queue, update its value to be the minima between
                            # its current value and this new distance
                            nodes[i][j].dist = min(dist + 1, nodes[i][j].dist)
                        elif (nodes[i][j].visited == False) or \
                        (dist + 1 < nodes[i][j].dist):
                            # this node is not in the queue, push it if it has not been visited or
                            # if our new distance is smaller than its recorded distance
                            nodes[i][j].dist = dist + 1
                            heapq.heappush(pq, nodes[i][j])
                            in_queue[i][j] = True

        return -1

def main():
    test_cases = {
        "1": {
            "grid": [[0,1],[1,0]],
            "expected": 2,
        },
        "2": {
            "grid": [[0,0,0],[1,1,0],[1,1,0]],
            "expected": 4,
        },
        "3": {
            "grid": [[1,0,0],[1,1,0],[1,1,0]],
            "expected": -1,
        },
        "4": {
            "grid": [[0,0,0,0,0,0,0,0,0],[0,1,1,0,0,0,0,0,0],[1,0,0,1,0,0,0,0,0],[0,1,0,0,1,1,0,0,1],[0,0,1,0,0,1,0,0,1],[0,1,0,1,0,0,1,1,0],[0,0,0,0,0,1,0,0,0],[0,1,0,1,0,0,1,0,0],[0,1,1,0,0,0,0,1,0]],
            "expected": 10,
        },
        "5": {
            "grid": [[0,0,1,0,0,0,0],[0,1,0,0,0,0,1],[0,0,1,0,1,0,0],[0,0,0,1,1,1,0],[1,0,0,1,1,0,0],[1,1,1,1,1,0,1],[0,0,1,0,0,0,0]],
            "expected": 10,
        },
        "6": {
            "grid": [[0,0,1,0,0,1,0,1,0],[0,0,0,0,0,0,0,0,0],[0,1,1,0,1,1,1,1,1],[0,0,0,1,0,0,0,0,0],[1,1,0,0,0,1,0,0,0],[1,0,1,0,0,1,0,0,1],[1,1,1,1,0,0,1,0,0],[1,0,0,1,0,0,1,1,1],[0,0,0,0,0,0,0,0,0]],
            "expected": 11,
        },
        "7": {
            "grid": [[0]],
            "expected": 1,
        },
    }

    solution = Solution()

    for tk, targs in test_cases.items():
        expected = targs.pop("expected", None)
        ret = solution.shortestPathBinaryMatrix(**targs, verbose=True)
        if expected is not None:
            passed = ret == expected
        else:
            passed = None
        print(f"test case {tk}: {targs}\nReturned: {ret}, Expected: {expected}\nPassed:{passed}\n")
main()


test case 1: {'grid': [[0, 1], [1, 0]]}
Returned: 2, Expected: 2
Passed:True

test case 2: {'grid': [[0, 0, 0], [1, 1, 0], [1, 1, 0]]}
Returned: 4, Expected: 4
Passed:True

test case 3: {'grid': [[1, 0, 0], [1, 1, 0], [1, 1, 0]]}
Returned: -1, Expected: -1
Passed:True

test case 4: {'grid': [[0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 1, 0, 0, 0, 0, 0, 0], [1, 0, 0, 1, 0, 0, 0, 0, 0], [0, 1, 0, 0, 1, 1, 0, 0, 1], [0, 0, 1, 0, 0, 1, 0, 0, 1], [0, 1, 0, 1, 0, 0, 1, 1, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 1, 0, 1, 0, 0, 1, 0, 0], [0, 1, 1, 0, 0, 0, 0, 1, 0]]}
Returned: 10, Expected: 10
Passed:True

test case 5: {'grid': [[0, 0, 1, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 1], [0, 0, 1, 0, 1, 0, 0], [0, 0, 0, 1, 1, 1, 0], [1, 0, 0, 1, 1, 0, 0], [1, 1, 1, 1, 1, 0, 1], [0, 0, 1, 0, 0, 0, 0]]}
Returned: 10, Expected: 10
Passed:True

test case 6: {'grid': [[0, 0, 1, 0, 0, 1, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 1, 0, 1, 1, 1, 1, 1], [0, 0, 0, 1, 0, 0, 0, 0, 0], [1, 1, 0, 0, 0, 1, 0, 0, 0], [1, 0, 1, 0,